In [4]:
#Compare process time when files are all distributed compared to just in 1 node?
#Same as untitled but will distribute the file to all nodes first and then executeimport findspark
#https://mapr.com/blog/tutorial-using-pyspark-and-mapr-sandbox/
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark

from collections import OrderedDict
from numpy import array
from math import sqrt
import sys
import os
import numpy
import urllib
from pyspark import SparkContext
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.linalg import SparseVector
from collections import OrderedDict
from time import time

conf = pyspark.SparkConf()
conf.setAppName('Sparkl Python RDD Basics')
conf.set("spark.cores.max", "16")
conf.set("spark.yarn.executor.memoryOverhead", "0")
conf.set("spark.yarn.executor.memory", "512M")
conf.set("spark.yarn.driver.memory", "512M")
conf.set("spark.submit.deployMode", "client")
# sc.stop()
# sc = pyspark.SparkContext(conf=conf)
sc = pyspark.SparkContext.getOrCreate(conf=conf)

In [5]:
#if file is in hduser, no need for file:///
data_file = "kddcup.data.gz"
kddcup_data = sc.textFile(data_file)

In [6]:
def parse_interaction(line):
    line_split = line.split(",")
    clean_line_split = [line_split[0]]+line_split[4:-1]
    return (line_split[-1], array([float(x) for x in clean_line_split]))

parsed_data = kddcup_data.map(parse_interaction)
pd_values = parsed_data.values().cache()

In [7]:
kdd_sample = pd_values.sample(False, .10, 123)

In [8]:
standardizer = StandardScaler(True, True)
standardizer_model = standardizer.fit(kdd_sample)

In [9]:
data_for_cluster = standardizer_model.transform(kdd_sample)

In [10]:
k_values = range(10,31,10)
metrics = []

k_values

range(10, 31, 10)

In [11]:
def error(point): 
    center = clusters.centers[clusters.predict(point)] 
    denseCenter = DenseVector(numpy.ndarray.tolist(center)) 
    return sqrt(sum([x**2 for x in (DenseVector(point.toArray()) - denseCenter)]))

In [16]:
import time

t0 = time.time()

clusters = KMeans.train(data_for_cluster, 30, maxIterations=4, initializationMode="k-means||")
WSSSE = data_for_cluster.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print(30, WSSSE)

tt = time.time() - t0


print(tt)

30 306384.7358574851
3299.7056498527527
